<div>
    <p>Submission Details: Chang Feng (cf4ff)</p>
    <p>Collaboration Note: Kathy Wang (we discuss the problem of multicollinearity)</p>
    
</div>

## Project 1: Housing Price Prediction

<div class="alert alert-block alert-danger">
<b>Due: 9:29am, Tuesday, 22 January 2019</b>
</div>

Price prediction is one the key ingredients in market design and
market competition. An important feature of the price in a competitive
market is that it arises as an outcome of the market equilibrium where
supply is equal to demand. As a result, factors that may be good
predictors of the price, such as the volume of market sales, do not
have a causal relationship with it.

The distinction between factors that have good predictive power and
causal factors becomes particularly important when prediction is
needed in the changing market settings.  For instance, the change in
average disposable consumer income shifts market demand, causing the
change in both the prices and the volume of sales. Accounting for
market conditions, substitution between products and variation in
consumer demographics makes such predictions even more challenging in
complex markets, such as the market for real estate.

In your first project you are tasked with constructing a model for predicting the median price of a detached 3-bedroom single-family home in Charlottesville's zip code 22901. We provide some starting code to give you and example of how to analyze the zillow data. Then, you need to perform your own analysis to answer the given questions. Your answers should be in the form of a clear argument that includes both well-written prose and code and its results (when the notebook is run).

<div class="alert alert-block alert-info">
You and your assigned partner should work together on this
assignment. Both team members should fully understand everything you
submit.  If there are parts you understand quickly but are new to your
partner, it is your responsibility to explain them to your partner
until everyone understands.  If there are parts that your partner
understands quickly but that are new to you, it is your responsibility
to insist that your partner explains things to you until you
understand them well.
</div>

### Data

We will use data provided by Zillow: https://www.zillow.com/research/data/

<div class="alert alert-block alert-warning">
Download the data you need for this assignment from:
https://files.zillowstatic.com/research/public/Zip.zip
</div>

This file contains many `.csv` (comma-separated values) files, one for each metric. Unzip the file to extract the CSV files into a directory of your choice. In this template, we assume the files are in `Zip/` under the same directory as this notebook is located.

Not all the CSV files in the zip file are relevant for this project. Below is a list of files that are potentially useful (but feel free to use any data files you want if you believe they could be useful for prediction):

> Zip_Zhvi_3bedroom.csv, Zip_MedianListingPrice_3Bedroom.csv, MedianDailyListings_SSA_AllHomes_Zip.csv, MonthlyListings_SSA_AllHomes_Zip.csv, NewMonthlyListings_SSA_AllHomes_Zip.csv, Sale_Counts_Zip.csv, Zip_Median_PriceCut_Dollar_AllHomes.csv, Zip_MedianPctOfPriceReduction_AllHomes.csv, Zip_PctOfHomesDecreasingInValues_AllHomes.csv, Zip_PctOfHomesIncreasingInValues_AllHomes.csv, Zip_PctOfListingsWithPriceReductions_AllHomes.csv, Zip_PriceToRentRatio_AllHomes.csv

### Libraries

You will find it useful to install several relevant libraries for this project (which will also be useful for later projects). 

We recommend using these libraries (but you are welcome to use any open source libraries you prefer):

- [pandas](https://pandas.pydata.org/) (Python Data Analysis Library):
````
conda install pandas
````

- [numpy](http://www.numpy.org/) (if you installed Anaconda, this should already be installed; if not, follow the directions there)

- [StatsModels](https://www.statsmodels.org/stable/index.html) 
````
conda install statsmodels
````

<div class="alert alert-block alert-warning">
Both people on your team should set up the data and these libraries on your own machine, so you can each run things locally.  You should also decide on a way to share the `project1.ipynd` file (its up to you how to do this, but recommended options include using Dropbox, Google Drive, or a shared private github repository).
</div>

Import the libraries and give them abbreviated names:

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

We use data from the follwing list of ZIP codes (covering the region around Charlottesville):

In [4]:
ziplist = [22901,22945,22904,22936,22909,22987,22905,22906,22907,22908,22910,22903,22932,22911,22940,22902,22935,22924,22959,22943,22947,22965,22923,22968,22931,24431,22920,22946,22973,24441,22827,22974,22980,22723,22937,22972,24471,24469,22738,24590,22748,22938,24437,22709,22957,22840,22942,22846,22958,22969,22939,22963,22731,22952,22727,24486,22732,22841,24482,24467,24562,22832,23170,22949,24477,22848,22971,22989,22730,24401,23093,22801,23055,22807,23084,23022,22960,22711,22849,22948,24463,22715,23004,22722,22802,22851,22743,22812,24440,22967,22733,22719,22976,22833,23038,24581,22725,23921,24464,24599,24421,22735,23123,22729,24402,22922,22803,22964,24479,22567,23067,22834,24476,24472,22844,23117,22713,22701,22850,22542,22821,22843,23027,23936,23065,24553,23153,23063,22815,22740,22853,24533,22749,24411,22835,22433,24483,24459,22831,24430,23040,24485,24435,23024,24415,24521,22716,22847,22741,22508,22737,22726,22830,23014,24432,22714,24595,22650,22551,22746,23102,22718,22534,22842,22747,24473,22845,24442,22734,22820,23160,24416,23129,26802,23139,26815,24574,23039,22724,22736,20106,24439,22824,23192,22810,22553,22742,22610,22652,23015,24522,24572,23146,22627,22407,22712,23960,26838,24593,22811,24433,24458,22640,23103,22664,23939,20128,23909,23901,24538,22720,22565,26807,23966,26812,23958,22501,24513,24450,20186,24505,24506,24514,24515,22623,22639,22471,24555,24504,22728,24503,23083,22406,22412,24536,22739,23943,24460,22401,24487,23233,22660,22644,20116,20139,26810,20188,23238,24501,20138,24526,22580,22630,23047,23002,22403,22402,22404,23059,24468,23113,22626,22546,20187,22642,24578,24502,26866,24465,20119,20115,23005,22643,24579,22545,22408,23120,23060,24413,22446,23229,22556,23294,22657,22405,26851,23963,23058,24588,23114,22649,23288,26814,26804,24551,23934,23922,22428,22645,23173,22555,23112,23228,20185,20144,20137,23226,22463,22430,22654,22554,20181,26836,22547,24422,23942,23235,22655,23230,24556,20182,22641,26801,20140,23236,26847,26884,23221,24944,23227,22134,20198,24484,23923,26886,24448,23225,26855,23832,23959,20155,20136,24412,23242,24570,22663,23069,23218,23241,23249,23260,23261,23269,23273,23274,23276,23278,23279,23282,23284,23285,23286,23290,23291,23292,23293,23295,23930,24915,23220,23105,24550,26818,23954,22535,22427,20156,23232,23289,24920,22538,23222,23955,26823,23219,24066,22514,24934,23224,20112,23298,24554,22552,22646,22172,24528,24576,22620,20130,20168,24438,20169,20118,24474,23947,20184,20110,23116,20109,23838,22451,26296,23255,23234,23297,24457,20108,22025,23223,22602,24523,20113,22135,24445,20143,26263,23162]

### Example Analysis

We choose two consecutive months arbitrariliy. We use the first month's data to fit our regression model, and then test our model's performance on data from the next month. Here, we use August 2018 (2018-08) and September 2018 (2018-09). But you can use any two consecutive months in the database.

In [5]:
month1 = '2018-08'
month2 = '2018-09'

As an example, here we use the ZHVI (Zillow Home Value Index) data from the `Zip_Zhvi_3bedroom.csv` file:

In [19]:
df_zhvi = pd.read_csv('Zip/Zip_Zhvi_3bedroom.csv') # load the ZHVI data, use the directory where you saved the data
df_zhvi = df_zhvi[['RegionName', month1, month2]] # keep only the columns we want; in this case RegionName (ZIP), month1 and month2
df_zhvi = df_zhvi.rename(columns={'RegionName': 'zip', month1: 'ZHVI1', month2: 'ZHVI2'}) # rename column names for easy reference later
df_zhvi = df_zhvi[df_zhvi['zip'].isin(ziplist)]  # we only keep the data from the relevant ZIP codes

Load a few more predictive variables as above. Then load the file with the median listing price:

In [9]:
df_price = pd.read_csv('Zip/Zip_MedianListingPrice_3Bedroom.csv', encoding='ISO-8859-1') 
df_price = df_price[['RegionName', month1, month2]]
df_price = df_price.rename(columns={'RegionName': 'zip', month1: 'price1', month2: 'price2'})
df_price = df_price[df_price['zip'].isin(ziplist)]

Now, merge all the dataframes into one dataframe:

In [10]:
df_merged = pd.merge(df_zhvi, df_price, on=['zip']) # merge on zipcode

Split the dataframe into `month1` and `month2`, and for each month, create an independent variable (`x`) with the predictive variables, and a dependent variable (`y`) with the price.

In [11]:
x_month1 = df_merged[['ZHVI1']]  # if you have multiple variables, list all of them. Here we only have one ZVHI1.
y_month1 = df_merged[['price1']] # the prices for month1

x_month2 = df_merged[['ZHVI2']]  # for month2
y_month2 = df_merged[['price2']] # prices for month2

Now fit the linear regression model on `month1`'s data:

In [12]:
model = sm.OLS(y_month1, x_month1).fit()  # fit the model
print(model.summary())      # print a summary of results

                            OLS Regression Results                            
Dep. Variable:                 price1   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     5139.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):           4.20e-62
Time:                        21:55:42   Log-Likelihood:                -753.35
No. Observations:                  64   AIC:                             1509.
Df Residuals:                      63   BIC:                             1511.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ZHVI1          1.1513      0.016     71.685      0.0

Predict `month2`'s prices using the model fitted above:

In [15]:
yhat = model.predict(x_month2)

Calculate mean squared error (MSE), variance and r-squared:

In [16]:
# MSE
serror = np.square(y_month2.price2 - yhat)
mse = np.mean(serror)
print("MSE: ", mse)

# Sample Variance
ybar = y_month2.price2.mean()
variance = np.mean((y_month2.price2 - ybar)**2)
print("Variance: ", variance)

# Explanation of variance
# this is the r-squared 
rsq = 1 - (mse / variance)
print("R-squared: ", rsq)

MSE:  936069904.3774337
Variance:  4173179536.05249
R-squared:  0.775693833373178


### Assignment

You should complete the assignment by inserting cells in the notebook with your answers to these questions, including both prose and code you used for your analysis.

<div class="alert alert-block alert-warning">
Construct and estimate a linear regression model that uses variables from the zip codes in the 75-mile radius of 22901, inventory variables from the zip code 22901 but none of the price variables from that zip code in a given month to predict the price of a 3-bedroom single-family home in 22901 next month.  
</div>

Provide an argument which variables can potentially be good predictors for the price of interest and try to estimate the linear regression with all those variables included. Discuss which models you have considered estimating but decided to discard and why. Present and discuss the outcome of that estimation.  


   

<div class="alert alert-block alert-info">
    
<p>There are many good choices for variables. I think the price of houses can be broken down into two parts, one is the value of the house, the other is the residual. </p>

<p>To predict the intrinsic value of the house, I looked into monthly listings to represent market supply for houses and sales count as market demand for houses, the Zillow Housing Value Index should be an excellent indicator of housing value as well. </p>

<p>Then for the residual, I thought the best predictor would be price-to-rent ratio which hopefully will indicate how much price deviate from its intrinsic value. Other good predictors would be able to show how much price reduction occurs from listing sales which I chose median percentage of price reductions as indicator of that. </p>

<p>This is the logic behind model 1, after the regression however, I discovered that there is multicollinearity problem with this model. After some experimentation, it seemed that Zillow Housing Value Index is correlated with most of the variables I was using, which means that this index might be calculated from those variables. Therefore those variables should be dropped. </p>
    
<p>In the second model, I only used the ZHVI as estimate of house value and median price cut as estimate of the  residual, and it solved the multicollinearity problem. Even though it has a lower R-squared, I eventually chose this model because the coefficients should be more accurate without multicollinearity. </p>

</div>

In [70]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

ziplist = [22901,22945,22904,22936,22909,22987,22905,22906,22907,22908,22910,22903,22932,22911,22940,22902,22935,22924,22959,22943,22947,22965,22923,22968,22931,24431,22920,22946,22973,24441,22827,22974,22980,22723,22937,22972,24471,24469,22738,24590,22748,22938,24437,22709,22957,22840,22942,22846,22958,22969,22939,22963,22731,22952,22727,24486,22732,22841,24482,24467,24562,22832,23170,22949,24477,22848,22971,22989,22730,24401,23093,22801,23055,22807,23084,23022,22960,22711,22849,22948,24463,22715,23004,22722,22802,22851,22743,22812,24440,22967,22733,22719,22976,22833,23038,24581,22725,23921,24464,24599,24421,22735,23123,22729,24402,22922,22803,22964,24479,22567,23067,22834,24476,24472,22844,23117,22713,22701,22850,22542,22821,22843,23027,23936,23065,24553,23153,23063,22815,22740,22853,24533,22749,24411,22835,22433,24483,24459,22831,24430,23040,24485,24435,23024,24415,24521,22716,22847,22741,22508,22737,22726,22830,23014,24432,22714,24595,22650,22551,22746,23102,22718,22534,22842,22747,24473,22845,24442,22734,22820,23160,24416,23129,26802,23139,26815,24574,23039,22724,22736,20106,24439,22824,23192,22810,22553,22742,22610,22652,23015,24522,24572,23146,22627,22407,22712,23960,26838,24593,22811,24433,24458,22640,23103,22664,23939,20128,23909,23901,24538,22720,22565,26807,23966,26812,23958,22501,24513,24450,20186,24505,24506,24514,24515,22623,22639,22471,24555,24504,22728,24503,23083,22406,22412,24536,22739,23943,24460,22401,24487,23233,22660,22644,20116,20139,26810,20188,23238,24501,20138,24526,22580,22630,23047,23002,22403,22402,22404,23059,24468,23113,22626,22546,20187,22642,24578,24502,26866,24465,20119,20115,23005,22643,24579,22545,22408,23120,23060,24413,22446,23229,22556,23294,22657,22405,26851,23963,23058,24588,23114,22649,23288,26814,26804,24551,23934,23922,22428,22645,23173,22555,23112,23228,20185,20144,20137,23226,22463,22430,22654,22554,20181,26836,22547,24422,23942,23235,22655,23230,24556,20182,22641,26801,20140,23236,26847,26884,23221,24944,23227,22134,20198,24484,23923,26886,24448,23225,26855,23832,23959,20155,20136,24412,23242,24570,22663,23069,23218,23241,23249,23260,23261,23269,23273,23274,23276,23278,23279,23282,23284,23285,23286,23290,23291,23292,23293,23295,23930,24915,23220,23105,24550,26818,23954,22535,22427,20156,23232,23289,24920,22538,23222,23955,26823,23219,24066,22514,24934,23224,20112,23298,24554,22552,22646,22172,24528,24576,22620,20130,20168,24438,20169,20118,24474,23947,20184,20110,23116,20109,23838,22451,26296,23255,23234,23297,24457,20108,22025,23223,22602,24523,20113,22135,24445,20143,26263,23162]

month1 = '2018-09'
month2 = '2018-10'

df_zhvi = pd.read_csv('Zip/Zip_Zhvi_3bedroom.csv') 
df_zhvi = df_zhvi[['RegionName', month1, month2]] 
df_zhvi = df_zhvi.rename(columns={'RegionName': 'zip', month1: 'ZHVI1', month2: 'ZHVI2'}) 
df_zhvi = df_zhvi[df_zhvi['zip'].isin(ziplist)] 

df_ptr = pd.read_csv('Zip/Zip_PriceToRentRatio_AllHomes.csv',encoding='ISO-8859-1')
df_ptr = df_ptr[['RegionName', month1, month2]]
df_ptr = df_ptr.rename(columns={'RegionName': 'zip', month1: 'ptr1', month2: 'ptr2'})
df_ptr = df_ptr[df_ptr['zip'].isin(ziplist)]

df_pctLpReduc = pd.read_csv('Zip/Zip_PctOfListingsWithPriceReductions_AllHomes.csv',encoding='ISO-8859-1') 
df_pctLpReduc = df_pctLpReduc[['RegionName', month1, month2]] 
df_pctLpReduc = df_pctLpReduc.rename(columns={'RegionName': 'zip', month1: 'pctLpReduc1', month2: 'pctLpReduc2'}) 
df_pctLpReduc = df_pctLpReduc[df_pctLpReduc['zip'].isin(ziplist)] 

df_MedPctPrReduc = pd.read_csv('Zip/Zip_MedianPctOfPriceReduction_AllHomes.csv',encoding='ISO-8859-1') 
df_MedPctPrReduc = df_MedPctPrReduc[['RegionName', month1, month2]] 
df_MedPctPrReduc = df_MedPctPrReduc.rename(columns={'RegionName': 'zip', month1: 'MedPctPrReduc1', month2: 'MedPctPrReduc2'}) 
df_MedPctPrReduc = df_MedPctPrReduc[df_MedPctPrReduc['zip'].isin(ziplist)] 

df_MonthList = pd.read_csv('Zip/MonthlyListings_SSA_AllHomes_Zip.csv',encoding='ISO-8859-1') 
df_MonthList = df_MonthList[['RegionName', month1, month2]] 
df_MonthList = df_MonthList.rename(columns={'RegionName': 'zip', month1: 'MonthList1', month2: 'MonthList2'}) 
df_MonthList = df_MonthList[df_MonthList['zip'].isin(ziplist)] 

df_SalesCount = pd.read_csv('Zip/Sale_Counts_Zip.csv',encoding='ISO-8859-1') 
df_SalesCount = df_SalesCount[['RegionName', month1, month2]] 
df_SalesCount = df_SalesCount.rename(columns={'RegionName': 'zip', month1: 'SalesCount1', month2: 'SalesCount2'}) 
df_SalesCount = df_SalesCount[df_SalesCount['zip'].isin(ziplist)] 


df_price = pd.read_csv('Zip/Zip_MedianListingPrice_3Bedroom.csv', encoding='ISO-8859-1') 
df_price = df_price[['RegionName', month1, month2]]
df_price = df_price.rename(columns={'RegionName': 'zip', month1: 'price1', month2: 'price2'})
df_price = df_price[df_price['zip'].isin(ziplist)]

df_MedDailyList = pd.read_csv('Zip/MedianDailyListings_SSA_AllHomes_Zip.csv', encoding='ISO-8859-1') 
df_MedDailyList = df_MedDailyList[['RegionName', month1, month2]]
df_MedDailyList = df_MedDailyList.rename(columns={'RegionName': 'zip', month1: 'MedDailyList1', month2: 'MedDailyList2'})
df_MedDailyList = df_MedDailyList[df_MedDailyList['zip'].isin(ziplist)]


df_medPrCut = pd.read_csv('Zip/Zip_Median_PriceCut_Dollar_AllHomes.csv', encoding='ISO-8859-1') 
df_medPrCut = df_medPrCut[['RegionName', month1, month2]]
df_medPrCut = df_medPrCut.rename(columns={'RegionName': 'zip', month1: 'MedPrCut1', month2: 'MedPrCut2'})
df_medPrCut = df_medPrCut[df_medPrCut['zip'].isin(ziplist)]

df_PctHomeDecrVal = pd.read_csv('Zip/Zip_PctOfHomesDecreasingInValues_AllHomes.csv', encoding='ISO-8859-1') 
df_PctHomeDecrVal = df_PctHomeDecrVal[['RegionName', month1, month2]]
df_PctHomeDecrVal = df_PctHomeDecrVal.rename(columns={'RegionName': 'zip', month1: 'PctHomeDecrVal1', month2: 'PctHomeDecrVal2'})
df_PctHomeDecrVal = df_PctHomeDecrVal[df_PctHomeDecrVal['zip'].isin(ziplist)]


df_merged = pd.merge(df_zhvi, df_ptr, on=['zip'])
df_merged = pd.merge(df_merged, df_pctLpReduc, on=['zip'])
df_merged = pd.merge(df_merged, df_MedPctPrReduc, on=['zip'])
df_merged = pd.merge(df_merged, df_MonthList, on=['zip'])
df_merged = pd.merge(df_merged, df_SalesCount, on=['zip'])
df_merged = pd.merge(df_merged, df_price, on=['zip'])
df_merged = pd.merge(df_merged, df_medPrCut, on=['zip'])
df_merged = pd.merge(df_merged, df_PctHomeDecrVal, on=['zip'])
df_merged = pd.merge(df_merged, df_MedDailyList, on=['zip'])


#model 1
model1_x_month1 = df_merged[['ZHVI1','ptr1','MedPctPrReduc1','MonthList1','SalesCount1']] 
y_month1 = df_merged[['price1']] #prices for month1

model1_x_month2 = df_merged[['ZHVI2','ptr2','MedPctPrReduc2','MonthList2','SalesCount2']]
y_month2 = df_merged[['price2']] # prices for month2

print('model 1: ')
model1 = sm.OLS(y_month1, model1_x_month1).fit()
print(model1.summary())

#model 2
model2_x_month1 = df_merged[['ZHVI1','MedPrCut1']] 
#y_month1 = df_merged[['price1']] #prices for month1

model2_x_month2 = df_merged[['ZHVI2','MedPrCut2']]
#y_month2 = df_merged[['price2']] # prices for month2

print()
print('model 2: ')
model2 = sm.OLS(y_month1, model2_x_month1).fit()
print(model2.summary())



model 1: 
                            OLS Regression Results                            
Dep. Variable:                 price1   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     1194.
Date:                Mon, 21 Jan 2019   Prob (F-statistic):           6.03e-19
Time:                        11:29:46   Log-Likelihood:                -222.24
No. Observations:                  20   AIC:                             454.5
Df Residuals:                      15   BIC:                             459.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
ZHVI1              0.7766      0.1

Compute the in-sample mean squared error of your prediction, i.e. the sum of squared deviations of the price predicted by your model and the price that was actually observed in the data. Compare the mean-squared error of your model with the empirical variance of the price of 3-bedroom houses in zip code 22901. How much of that price variance is explained by your model in percentage terms?


<div class="alert alert-block alert-info"> 
    <p>In-sample mean squared error is 598,361,239. Total Variance is 5,521,466,534. Therefore, 89% of the variation is explained by model 2. </p>
</div>

In [71]:
#model 1
#print("Model 1: ")
yhat1 = model1.predict(model1_x_month2)
# MSE
serror1 = np.square(y_month2.price2 - yhat1)
mse1 = np.mean(serror1)
#print("MSE: ", mse1)

# Sample Variance
ybar = y_month2.price2.mean()
variance = np.mean((y_month2.price2 - ybar)**2)
print("Variance: ", variance)

# Explanation of variance
# this is the r-squared 
rsq1 = 1 - (mse1 / variance)
#print("R-squared: ", rsq1)


#model 2
print("Model 2: ")
yhat2 = model2.predict(model2_x_month2)
# MSE
serror2 = np.square(y_month2.price2 - yhat2)
mse2 = np.mean(serror2)
print("MSE: ", mse2)

# Explanation of variance
# this is the r-squared 
rsq2 = 1 - (mse2 / variance)
print("R-squared: ", rsq2)


Variance:  5521466534.9475
Model 2: 
MSE:  598361239.2174904
R-squared:  0.8916300161505588


How does your analysis relate to the idea of generalization performance discussed in the introduction to V. Vapnik's _The Nature of Statistical Learning Theory_?


<div class="alert alert-block alert-info">
    <p>Generalization performance is about how well the model perform on the training set compared to its performance on the prediction.</p>
    <p>As a rough estimate on the performance of my model, I simply calculated the r-squared value of using the model predicting the training set (Sep 2018 prices). The result is 91% of the variation in Sep 2018 prices is explained by my model. As opposed to 89% of variation in Oct 2018 prices explained by the model, it is not a big difference. Therefore, I think the model has a pretty good generalization perfomance. </p>
</div>

In [78]:
yhat_tr = model2.predict(model2_x_month1)
serror_tr = np.square(y_month1.price1 - yhat_tr)
mse_tr = np.mean(serror_tr)

ybar_tr = y_month1.price1.mean()
variance = np.mean((y_month1.price1 - ybar_tr)**2)

rsq_tr = 1 - (mse_tr / variance)
print("Training set R-squared: ", rsq_tr)

Training set R-squared:  0.9139561852492093
